In [ ]:
from typing import List

import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer


# можете сменить на mps на макбуке, но лично у меня он криво работает
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Знакомство с Transformers - 35 баллов

## Создание модели и предсказание следующего токена - 5 баллов
Нужно создать модель через `AutoModelForCausalLM`, создать токенайзер через `AutoTokenizer` и олучить следующий токен через жадную генерацию!

**Внимание** на каких-то из функций далее у вас может кончаться видеопамять из-за хранения активаций. Чтобы этого не происходило рекомендуется все вычисления оборачивать в контекстный менеджер `with torch.no_grad()`

In [ ]:
model_name = "openai-community/gpt2"
# Загружаем модель для причинного языкового моделирования
model = AutoModelForCausalLM.from_pretrained(model_name)
# Загружаем соответствующий токенайзер
tokenizer = AutoTokenizer.from_pretrained(model_name)


text = "This is a sample text"

# Нужно преобразовать text с помощью tokenizer() и подать это в model.forward() (он же просто model())
# после этого мы получим logits [batch_size = 1, seq_len, d_model]
# По этому тензору нужно предсказать следующее слово!

# Преобразуем текст в токены и создаем тензор ввода
inputs = tokenizer(text, return_tensors="pt")

# Отключаем вычисление градиентов для экономии видеопамяти
with torch.no_grad():
    # Пропускаем вход через модель и получаем логиты
    outputs = model(**inputs)
    logits = outputs.logits

# Логиты имеют размер [batch_size, seq_len, vocab_size]
# Мы берем логиты последнего токена в последовательности
last_token_logits = logits[:, -1, :]
next_token_idx = last_token_logits.argmax(dim=-1).item()
# Декодируем индекс токена обратно в строку
next_token = tokenizer.decode([next_token_idx])


next_token = tokenizer.decode([next_token_idx])

assert next_token.strip() == "file"



## Используем Generate - 5 баллов

Мы с вами помним про различные виды сэмплинга - top_k, top_p, temperature,frequency penalty.
Отличная новость заключается в том, что нам не нужно все это писать самим! Оно уже включено в [GenerationMixin](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#generation), от которого наследуются модели для генерации текста.

Для генерации есть функция [generate](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#transformers.GenerationMixin.generate)

Ваша задача написать для модели выше генерацию по тексту с:
* Температурой - 0.9
* Top-K - 20
* Repetition Penalty (Frequency Penalty) - 1.2
* максимальное число новых токенов - 10


In [ ]:
text = "This is still a sample text, but"

# Преобразуем текст в токены и создаем тензор ввода
inputs = tokenizer(text, return_tensors="pt").to(device)

# Список для хранения результатов генерации
results = []

# Отключаем вычисление градиентов для экономии видеопамяти
with torch.no_grad():
  for i in range(10):
      # Генерация текста с заданными параметрами сэмплинга
      gens = model.generate(
            inputs.input_ids,
            do_sample=True,               # Включаем сэмплинг для разнообразия
            temperature=0.9,              # Температура сэмплинга
            top_k=20,                     # Top-K сэмплинг
            repetition_penalty=1.2,       # Штраф за повторение
            max_new_tokens=10,            # Максимум новых токенов
            pad_token_id=tokenizer.eos_token_id  # Идентификатор токена заполнения
        )
              # Декодируем сгенерированные токены обратно в строку
      generation = tokenizer.decode(gens[0], skip_special_tokens=True) # сгенерированный текст
      results.append(generation)

assert len(set(results)) > 1, "Все генерации получились одинаковыми, проверьте опции генерации и флаг do_sample!"

## Generate Batched - 5
Теперь давайте жадно сгенерируем текст, но забатчуем несколько сэмплов. До этого мы всегда генерировали по батчу размера 1, поэтому у нас не было паддингов!

Когда появляется несколько текстов разной длины, то появляются и паддинги.

Представим себе ситуцию, что у нас батч из двух элементов длины 2 и 5 (токен -1 будет выступать в качестве паддинга **только для удобства визуализации**).

Тогда

```python
input_ids = [
    [3, 2, -1, -1, -1]
    [5, 6,  7,  1,  2]
]
attention_mask = [
    [1, 1, 0, 0, 0],
    [1, 1, 1, 1, 1]
]
```

Представим, что мы сгенерировали еще один токен, тогда

```python
input_ids = [
    [3, 2, -1, -1, -1, 7]
    [5, 6,  7,  1,  2, 8]
]
attention_mask = [
    [1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1]
]
```

Получается, что у нас паддинги в маске возникают посередине. Мы не будем заниматься реализацией своего алгоритма генерации здесь, но отметим, что добавление паддинга слева значительно упрощает этот процесс.
Тогда исходная последовательность будет:

```python
input_ids = [
    [-1, -1, -1, 3, 2]
    [ 5,  6,  7, 1, 2]
]
attention_mask = [
    [0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1]
]
```

и после генерации следующего токена

```python
input_ids = [
    [-1, -1, -1, 3, 2, 7]
    [ 5,  6,  7, 1, 2, 8]
]
attention_mask = [
    [0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]
```

В качестве задания давайте соберем батч с левым паддингом и проверим, что жадная генерация (10 токенов) совпадает с генерацией на текстах по отдельности!

Для этого нам придется использовать параметр padding_side в конструкторе токенизатора.

In [ ]:
# Загружаем токенайзер с левым паддингом
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
tokenizer.pad_token_id = tokenizer.eos_token_id  # Устанавливаем pad_token_id

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Переносим модель на доступное устройство (GPU, если доступно)
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
texts = ["This is a sample text", "I'm really tired and this is just about"]

# Внимание! В данном задании нужна жадная генерация!

# Соберите оба текста в один батч и положите результаты генерации в
# batched_generations
batched_generations: List[str] = []

# Отключаем вычисление градиентов для экономии видеопамяти
with torch.no_grad():
    # ----------- Батч генерация ----------- #
    # Преобразуем тексты в токены с левым паддингом
    batch_inputs = tokenizer(texts, return_tensors="pt", padding=True).to(device)

    # Выполняем жадную генерацию (без сэмплинга)
    batch_outputs = model.generate(
        input_ids=batch_inputs.input_ids,
        attention_mask=batch_inputs.attention_mask,
        do_sample=False,            # Жадная генерация
        max_new_tokens=10,          # Максимум новых токенов
        pad_token_id=tokenizer.eos_token_id  # Идентификатор токена заполнения
    )

    # Декодируем сгенерированные токены обратно в строки
    batched_generations = tokenizer.batch_decode(batch_outputs, skip_special_tokens=True)



    # Пройдитесь по каждому сэмплу по отдельности и положите результаты генерации
    # в single_generations
    single_generations: List[str] = []
    # ----------- Индивидуальная генерация ----------- #
    for text in texts:
        # Токенизируем отдельный текст с левым паддингом
        single_input = tokenizer(text, return_tensors="pt", padding='longest').to(device)

        # Выполняем жадную генерацию
        single_output = model.generate(
            input_ids=single_input.input_ids,
            attention_mask=single_input.attention_mask,
            do_sample=False,            # Жадная генерация
            max_new_tokens=10,          # Максимум новых токенов
            pad_token_id=tokenizer.eos_token_id  # Идентификатор токена заполнения
        )

        # Декодируем сгенерированные токены обратно в строку
        generated_text = tokenizer.decode(single_output[0], skip_special_tokens=True)
        single_generations.append(generated_text)



    assert len(batched_generations) == 2 and len(single_generations) == 2
    for s, b in zip(batched_generations, single_generations):
        assert s == b

# KV Cache - 10 баллов
При генерации есть опция `use_cache` - это использование KV cache для генерации. Посмотреть визуализации про kv cache можно [тут](https://medium.com/@joaolages/kv-caching-explained-276520203249), или [тут](https://developer.nvidia.com/blog/mastering-llm-techniques-inference-optimization/#key-value_caching), или [тут](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/appnotes/transformers-neuronx/generative-llm-inference-with-neuron.html#kv-caching) или в лекции 4.

В рамках этой техники в при генерации в декодере считается только аттеншн последнего токена по всем векторам предыдущих токенов, которые посчитали на предыдущих этапах, а для "старых" (левых) токенов аттеншн не пересчитывается, т.к. "новые" (правые) токены на них не влияют.

Мы сохраняем K,V проекции предыдущих токенов (не пересчитываем $X W_K$ и $X W_V$) и считаем аттеншен только последнего токена по всем предыдущим. Аттеншены для предыдущих токенов мы не считаем, т.к. они для генерации не нужны, их выходы уже сохранениы в KV cache.

В рамках данного задания нужно:
1. Посчитать скорость генерации 100 токенов с и без kv cache, сказать, какая техника и во сколько раз быстрее.
2. Подсчитать скорость генерации 1 токена с и без kv cache, сказать, какая техника быстрее и почему.

Чтобы корректно сравнивать время генерации нужно использовать жадный сэмплинг!

**Ответы на оба вопроса нужно оставить письменно прямо здесь**.

<<Место для ответа>>
  - Среднее время генерации 100 токенов с KV Cache: 14.8756 секунд
  - Среднее время генерации 100 токенов без KV Cache: 241.5391 секунд
  - Отношениее в 16.0 раз

In [ ]:
# Функция для измерения времени генерации
def measure_generation_time(model, tokenizer, input_text, max_new_tokens, use_cache, num_runs=1):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    total_time = 0.0

    with torch.no_grad():
        for _ in range(num_runs):
            start_time = time.perf_counter()
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=max_new_tokens,
                do_sample=False,       # Жадная генерация
                use_cache=use_cache,   # Использование KV Cache
                pad_token_id=tokenizer.pad_token_id
            )
            if device.type == 'cuda':
                torch.cuda.synchronize()  # Ожидание завершения всех операций на GPU
            end_time = time.perf_counter()
            total_time += (end_time - start_time)

    average_time = total_time / num_runs
    return average_time


# Функция для измерения времени генерации 1 токена за раз
def measure_single_token_generation_time(model, tokenizer, input_text, num_tokens, use_cache):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    generated_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    total_time = 0.0

    with torch.no_grad():
        for _ in range(num_tokens):
            start_time = time.perf_counter()
            outputs = model.generate(
                input_ids=generated_ids,
                attention_mask=attention_mask,
                max_new_tokens=1,
                do_sample=False,       # Жадная генерация
                use_cache=use_cache,   # Использование KV Cache
                pad_token_id=tokenizer.pad_token_id
            )
            if device.type == 'cuda':
                torch.cuda.synchronize()  # Ожидание завершения всех операций на GPU
            end_time = time.perf_counter()
            total_time += (end_time - start_time)

            # Обновляем input_ids и attention_mask для следующей итерации
            generated_ids = outputs
            attention_mask = torch.ones_like(generated_ids)

    average_time = total_time / num_tokens
    return average_time

In [ ]:
import time
import numpy as np

text = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum lorem justo, semper dignissim ipsum vitae, sollicitudin aliquet eros. Duis id ultricies erat. Vivamus commodo auctor massa ut mollis. Maecenas lacinia tempus orci, imperdiet ullamcorper felis accumsan et. Etiam mattis neque diam, at egestas nunc eleifend id. Fusce tristique orci nec sollicitudin elementum. Nullam dui est, feugiat ac pellentesque at, posuere non massa.

Suspendisse accumsan ullamcorper dolor sed dictum. Mauris quis varius felis, quis gravida odio. Vestibulum diam arcu, aliquet convallis congue non, rutrum non turpis. Fusce vel orci ac diam suscipit lacinia. Curabitur maximus orci a dui gravida, accumsan convallis libero ornare. Phasellus dapibus, sapien pulvinar lacinia dictum, massa lacus scelerisque tellus, eu porta dolor eros vitae ex. Maecenas maximus, urna id pharetra dictum, dolor lorem sollicitudin ipsum, sit amet vestibulum orci felis quis leo. Pellentesque vel ligula ut urna eleifend condimentum nec et sem. Integer ligula nunc, rutrum ultricies urna et, congue suscipit lectus.
""".strip()



# Количество токенов для генерации
num_tokens = 100

# Подсчитаейте время генерации 100 новых токенов с помощью жадного сэмплирования при включенном KV Cache, выведите среднее время работы
time_with_cache = measure_generation_time(
    model=model,
    tokenizer=tokenizer,
    input_text=text,
    max_new_tokens=num_tokens,
    use_cache=True,
    num_runs=1  # к-во раз для более точного измерения
)

print(f"Среднее время генерации {num_tokens} токенов с KV Cache: {time_with_cache:.4f} секунд")


# Подсчитаейте время генерации 100 новых токенов с помощью жадного сэмплирования при выключенном KV Cache, выведите среднее время работы
time_without_cache = measure_generation_time(
    model=model,
    tokenizer=tokenizer,
    input_text=text,
    max_new_tokens=num_tokens,
    use_cache=False,
   num_runs=1  # к-во раз для более точного измерения
)

print(f"Среднее время генерации {num_tokens} токенов без KV Cache: {time_without_cache:.4f} секунд")

# Не забудьте ответить на вопросы вы описании задания!


Среднее время генерации 100 токенов с KV Cache: 14.8756 секунд
Среднее время генерации 100 токенов без KV Cache: 241.5391 секунд


In [ ]:
print(f"Отношениее в {time_without_cache//time_with_cache} раз")

Отношениее в 16.0 раз


In [ ]:
text = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum lorem justo, semper dignissim ipsum vitae, sollicitudin aliquet eros. Duis id ultricies erat. Vivamus commodo auctor massa ut mollis. Maecenas lacinia tempus orci, imperdiet ullamcorper felis accumsan et. Etiam mattis neque diam, at egestas nunc eleifend id. Fusce tristique orci nec sollicitudin elementum. Nullam dui est, feugiat ac pellentesque at, posuere non massa.

Suspendisse accumsan ullamcorper dolor sed dictum. Mauris quis varius felis, quis gravida odio. Vestibulum diam arcu, aliquet convallis congue non, rutrum non turpis. Fusce vel orci ac diam suscipit lacinia. Curabitur maximus orci a dui gravida, accumsan convallis libero ornare. Phasellus dapibus, sapien pulvinar lacinia dictum, massa lacus scelerisque tellus, eu porta dolor eros vitae ex. Maecenas maximus, urna id pharetra dictum, dolor lorem sollicitudin ipsum, sit amet vestibulum orci felis quis leo. Pellentesque vel ligula ut urna eleifend condimentum nec et sem. Integer ligula nunc, rutrum ultricies urna et, congue suscipit lectus.
""".strip()

# Подсчитаейте время генерации 1 нового токена с помощью жадного сэмплирования при включенном KV Cache, выведите среднее время работы
# Измерение времени генерации 1 токена с KV Cache (повторяем 100 раз)
time_single_with_cache = measure_single_token_generation_time(
    model=model,
    tokenizer=tokenizer,
    input_text=text,
    num_tokens=100,
    use_cache=True
)


print(f"Среднее время генерации 1 токена с KV Cache: {time_single_with_cache:.6f} секунд")



# Подсчитаейте время генерации 1 нового токена с помощью жадного сэмплирования при выключенном KV Cache, выведите среднее время работы
time_single_without_cache = measure_single_token_generation_time(
    model=model,
    tokenizer=tokenizer,
    input_text=text,
    num_tokens=100,
    use_cache=False
)

print(f"Среднее время генерации 1 токена без KV Cache: {time_single_without_cache:.6f} секунд")

# Не забудьте ответить на вопросы вы описании задания!
print(f"Отношениее в {time_single_without_cache//time_single_with_cache} раз")

Среднее время генерации 1 токена с KV Cache: 2.481780 секунд
Среднее время генерации 1 токена без KV Cache: 2.427287 секунд
Отношениее в 0.0 раз


# Скоринг, Perplixity - 6 баллов

Можно не только генерировать текст. Вспомним, что выдает после lm_head - вектор `[batch_size, seq_len, vocab_size]`, где для каждый вектор `[vocab_size]` это распределение вероятностей по следующему токену!

Опустим размерность batch_size=1 для удобства, seq_len = 4. Пусть у нас есть текст `bos мама мыла раму` (`bos` спецсимвол для начала текста)

Тогда вероятность этого текста расписывается через произведение условных вероятностей:

```
P(bos мама мыла раму) = P(мама | bos) * P(мыла | bos мама) * P(раму| bos мама мыла)
```

Т.е. это вероятность слова при условии его левого контекста.
Зачастую ее обозначают как $P(x_i|x_{<i})$ где $x_i$ - i-е слово, $x_{<i}$ - контекст $[x_1, x_2, x_3, ... x_{i-1}]$
Эти вероятности можно взять из выходного вектора!

Давайте попробуем подсчитать вероятность и perplexity текстов!
perplexity как и вероятность мера того насколько модель "уверена" в тексте, т.е. насколько по оценки ее параметрами данный текст вероятен.

$$Perplexity(X) = exp(-\frac {1} {N} \sum_{i}^{N} log P(x_i | x_{<i}))$$

В этом задании нужно:
1. Посчитать вероятность **text**
2. Посчитать перплексию **text**

Еще одна важная деталь:
работать с вероятностями плохо. Т.к. вероятность представляет собой число от 0 до 1, то при перемножении десятков или даже сотен таких числе теряется точность!
Для этого от произведения вероятностей берут логарифм и получают logprobs - логарифмы вероятностей. Их можно складывать, по свойству логарифма логарифм произведения равен произведению логарифма.

$$ p = p_1 * p_2 * p_3 $$
$$log(p) = log (p_1) + log (p_2) + log (p_3)$$
$$exp(log (p)) = p = exp(log (p_1) + log (p_2) + log (p_3)) = exp (log (p_1 * p_2 * p_3)) = p_1 * p_2 * p_3$$

В pytorch для этого есть `torch.log_softmax`, который считается численно стабильно!

In [ ]:
print(f"Beginning of sentence (BOS) token = `{tokenizer.bos_token}`")
print(f"End of sentence (EOS) token  = `{tokenizer.eos_token}`")
text = "<|endoftext|>I'm so very tired of this<|endoftext|>"

# Устанавливаем add_special_tokens=False, чтобы явно контролировать специальные токены
inputs = tokenizer(text, add_special_tokens=False, return_tensors='pt')

# Перенос входных данных на устройство
input_ids = inputs.input_ids.to(device)

with torch.no_grad():
    logits = model(input_ids).logits

    # ваш код здесь!
    # 1. Нужно обрезать logits по длине, т.к. для предсказаний по последнему токену нечего считать
    logits = logits[:, :-1, :]  # Форма: [1, seq_len - 1, vocab_size]
    # 2. Превращаем logits в log_probs
    log_probs = torch.log_softmax(logits, dim=-1)  # Форма: [1, seq_len - 1, vocab_size]

    # 3. Берем вероятности следующих токенов, т.к. по вектору i-й позиции мы предсказываем токен на позиции (i + 1)
    targets = input_ids[:, 1:]  # Форма: [1, seq_len - 1]
    # для этого нам поможет torch.gather
    log_probs_of_targets = log_probs.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)  # Форма: [1, seq_len -1]

    # 4. Считаем вероятности и perplexity!
    sum_log_probs = log_probs_of_targets.sum()  # Скаляр

    # Вычисляем вероятность текста путем экспоненцирования суммы логарифмов
    probability = torch.exp(sum_log_probs).item()

    # Вычисляем перплексию по формуле
    N = targets.size(1)  # Количество предсказаний
    perplexity = np.exp(-sum_log_probs.item() / N)

# должно получиться что-то около 2.1783e-14 для вероятности и около 51 для ppl
# Вывод результатов
print(f"Probability of the text: {probability:.6e}")
print(f"Perplexity of the text: {perplexity:.6f}")

Beginning of sentence (BOS) token = `<|endoftext|>`
End of sentence (EOS) token  = `<|endoftext|>`
Probability of the text: 2.178365e-14
Perplexity of the text: 51.019203


# Вопросы - 4 балла

**Ответьте на вопрсоы текстом прямо здесь!**


1. Какое значение P(X) вероятности текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.
2. Какое значение перплексии текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.



1 - Лучшее значение вероятности 1, что означает максимальную уверенность модели в тексте.  

2 - Лучшее значение перплексии также 1, что модель полностью уверена в предсказаниях каждого токена.

# Chat-Models

# Формат - 5 баллов
Как мы обсуждали на лекции, все chat-модели принимают входы в своем особом формате.
Он может быть описан текстом, а может быть заложен в шаблон, который доступен через `tokenizer.apply_chat_template`

In [ ]:
model_name = "EleutherAI/gpt-neo-1.3B" #"NousResearch/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id  # Устанавливаем pad_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.half,          # Используем 16-битную точность для экономии памяти
    #low_cpu_mem_usage=True           # Оптимизация использования памяти на CPU
).to(device)
model.eval()  # Переводим модель в режим оценки

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTNeoBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj):

In [ ]:
def move_to_device(d):
    for k, v in d.items():
        d[k] = v.to(device)
    return d

Давайте посмотрим, как chat модель отработает на обычном тексте. Используйте для генерации сэмплинг и kv cache, выведите 5 результатов генерации.

In [ ]:
# Исходный текст для генерации
text = "hello how are you"

# Токенизируем входной текст
inputs = tokenizer(text, return_tensors='pt')
inputs = move_to_device(inputs)  # Переносим входные данные на устройство

# Генерируем и выводим 5 вариантов продолжения
for i in range(5):
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,       # Максимальное количество генерируемых токенов
            do_sample=True,          # Включаем сэмплинг
            temperature=0.7,         # Температура для сэмплинга (регулирует разнообразие)
            top_p=0.9,               # Ядерное сэмплирование
            num_return_sequences=1,  # Количество возвращаемых последовательностей
            use_cache=True           # Используем kv cache для ускорения
        )

    # Декодируем сгенерированные токены в текст
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Генерация {i+1}:")
    print(generated_text)
    print("===" * 5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Генерация 1:
hello how are you
all good how are you all
alright thank you for having
me here today I'm here today
with my good friend and fellow
writer and author of the book
"The Art of Writing" he is
currently writing his second


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Генерация 2:
hello how are you doing today?
I am doing good,
I am doing good,
I am doing good.
I am doing good,
I am doing good,
I am doing good.
I am doing good,
I am doing good


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Генерация 3:
hello how are you?
i'm fine
what's your name?
my name is jim
hello jim
how are you?
i'm fine
what's your name?
my name is jim
hello jim
how are you


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Генерация 4:
hello how are you
my name is lisa i'm a
fellow member of the
council of eu we're
a group of scientists
who study the origins
of life and
the history of life
and the origins of life

Генерация 5:
hello how are you?" "I'm fine." "I was just wondering if you had a minute?" "Sure, I can squeeze you in." "Okay." "Oh, God, I was just thinking." "It's weird, I'm not used to being


Видим, что текст зачастую выходит мусорный. Это потому что формат входных данных сильно отличается от того, что модель видела на обучении.
Как мы уже обсуждали, у всех chat-моделей свой формат. Где-то он описан просто словами, где-то он заложен в токенайзер. Мы рассмотрим как раз такой случай - за нас есть удобно написанная функция `apply_chat_template`. Давайте используем ее, чтобы получить префикс для генерации модели.

Не забудьте про опцию add_generation_prefix - она добавляет часть формата, после которой ожидается ответ модели!

In [ ]:
messages = [
    {"role": "user", "content": "hello"},
    {"role": "assistant", "content": "I'm good. How can I help you today"},
    {"role": "user", "content": "I love you"},
]

prefix = tokenizer.apply_chat_template(tokenizer, messages, add_generation_prefix=True)

reference = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm good. How can I help you today<|eot_id|><|start_header_id|>user<|end_header_id|>

I love you<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

assert prefix.strip() == reference.strip()

ValueError: Cannot use apply_chat_template() because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

Пробуем выкрутиться!!!!!

In [ ]:
from transformers import PreTrainedTokenizerBase

# Определяем собственный шаблон чата
class SimpleChatTemplate:
    def build_prompt(self, messages: List[dict], add_generation_prefix: bool = True) -> str:
        return format_messages(messages)

def format_messages(messages: List[dict]) -> str:
    """
    Форматирует список сообщений в строку-промпт для модели.

    Args:
        messages (List[dict]): Список сообщений, каждое из которых является словарём с ключами 'role' и 'content'.

    Returns:
        str: Форматированный строковый промпт.
    """
    formatted = ""
    for message in messages:
        role = message.get("role", "").capitalize()
        content = message.get("content", "")
        if role and content:
            formatted += f"{role}: {content}\n"
    # Добавляем следующую роль для генерации ответа
    formatted += "assistant:"
    return formatted


In [ ]:
# Присваиваем шаблон токенизатору
tokenizer.chat_template = SimpleChatTemplate()

messages = [
    {"role": "user", "content": "hello"},
    {"role": "assistant", "content": "I'm good. How can I help you today"},
    {"role": "user", "content": "I love you"},
]


# Теперь можно использовать apply_chat_template
prefix = tokenizer.chat_template.build_prompt(messages)


reference = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm good. How can I help you today<|eot_id|><|start_header_id|>user<|end_header_id|>

I love you<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

assert prefix.strip() == reference.strip()

AssertionError: 

Это не инструктивная модель и в ней нет нужных инструкций!

In [ ]:
prefix.strip() , reference.strip()

("User: hello\nAssistant: I'm good. How can I help you today\nUser: I love you\nassistant:",
 "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nhello<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI'm good. How can I help you today<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI love you<|eot_id|><|start_header_id|>assistant<|end_header_id|>")

Давайте посмотрим, что нам ответит модель!

In [ ]:
inputs = tokenizer(prefix, return_tensors="pt")

# Список для хранения результатов генерации
batched_generations: List[str] = []

# Генерация и вывод результатов
with torch.no_grad():
    for i in range(5):
        # Генерация текста
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,       # Максимальное количество генерируемых токенов
            do_sample=True,          # Включаем сэмплинг
            temperature=0.7,         # Температура для сэмплинга (регулирует разнообразие)
            top_p=0.9,               # Ядерное сэмплирование
            num_return_sequences=1,  # Количество возвращаемых последовательностей
            use_cache=True           # Используем kv cache для ускорения
        )
        # Декодирование сгенерированных токенов
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        batched_generations.append(generated_text)

        # Вывод сгенерированного текста
        print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: hello
Assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I help you today


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: hello
Assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I help you today


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: hello
Assistant: I'm good. How can I help you today
User: I love you
assistant: I love you too.
User: I'm not sure what you mean
assistant: I don't know what you mean.
User: I'm not sure what you mean
assistant: I don't know what you mean.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: hello
Assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I help you today
User: hello
Assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I help you today
User: I love you
assistant: I'm good. How can I product
User: I love you
assistant: I'm good. How can I product
User: I


-----------------

- Далее вероятно уже не успею, поэтому до основного дедлайна сдаю следующего пункта!

- Если найду время, то постараюсь доделать концовку к мягкому дедлайну.

## Benchmark - 15 баллов

Перед нами датасет MMLU - датасет вопросов и ответов в стиле multiple choice.
* question - вопрос
* choices - варианты ответа
* answer - номер правильного ответа

In [ ]:
from datasets import load_dataset
mmlu = load_dataset("cais/mmlu", "global_facts", split="test")
mmlu[1]

Наша задача здесь решить задачу многоклассовой классификации.
Для этого нужно посчитать
$$P(choices_i | question)$$
т.е. для посчитать вероятность каждого варианта ответа для вопроса. Мы это уже делали кодом выше!

После этого давайте брать самый вероятный ответ и считать, что модель его выбрала.
После этого давайте посчитаем accurracy, т.е. долю правильных ответов.
Вместо вероятностей для подсчета лучше использовать logprobs.

Итого, что нужно сделать:
1. Пройтись по датасету, для каждого question и каждого из соответствующих choices получить самый вероятный ответ.
2. Посчитать итоговый accuracy

* За базовую реализацию с двойным циклом по questions и choices дается 5 баллов.
* еще 3 балла дается, если собрать question и все его choices в батч размера 4 и считать вероятности параллельно для всего батча сразу.
* еще 3 балла дается, если собрать несколько questions и их choces в батч размера 4 * num_questions_in_batch и считать вероятности параллельно для всего батча сразу.

Что важно помнить:
1. Данные нужно подавать в правильном формате для llama3!
2. Если делаете варианты с батчеванием помните: длины choices могут быть разными! Нужно не считать вероятности по паддингам. В этом нам помогут attention_masks из выходов `tokenizer()`

In [ ]:
# ваш код здесь!

**Ответьте на следующие вопросы (2 балла за вопрос)**:
1. Как влияет длина ответа на вероятность ответа при скоринге? Если есть какие-либо проблемы, как бы вы с этим боролись.
2. Если к началу каждого ответа добавилить метки A) B) C) D) станет ли модель отвечать лучше или хуже?
Стоит ли по-вашему добавлять эти метки?
